In [10]:
import numpy as np
import random
import tkinter as tk
import time
import heapq
import tkinter.simpledialog as simpledialog

class Node:
    def __init__(self, position, parent=None):
        self.position = position
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def a_star_search(grid, start, goal):
    open_list = []
    closed_set = set()

    start_node = Node(start)
    goal_node = Node(goal)

    heapq.heappush(open_list, start_node)

    while open_list:
        current_node = heapq.heappop(open_list)

        if current_node.position == goal:
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]

        closed_set.add(current_node.position)

        for new_position in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])

            if (0 <= node_position[0] < len(grid)) and (0 <= node_position[1] < len(grid[0])):
                if grid[node_position[0]][node_position[1]] == 1 and node_position not in closed_set:
                    child_node = Node(node_position, current_node)
                    child_node.g = current_node.g + 1
                    child_node.h = heuristic(child_node.position, goal_node.position)
                    child_node.f = child_node.g + child_node.h

                    if add_to_open(open_list, child_node):
                        heapq.heappush(open_list, child_node)

    return None

def add_to_open(open_list, child_node):
    for node in open_list:
        if child_node.position == node.position and child_node.g > node.g:
            return False
    return True

class QLearningAgent:
    def __init__(self, agent_id, grid_size, obstacles, start, goal, initial_path, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.agent_id = agent_id
        self.grid_size = grid_size
        self.obstacles = obstacles
        self.start = start
        self.initial_start = start  # Store the initial start position
        self.goal = goal
        self.current_position = start
        self.initial_path = initial_path
        self.q_table = np.zeros((*grid_size, 5))  # 4 directions + 1 for staying in place
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.actions = [(0, 1), (1, 0), (0, -1), (-1, 0), (0, 0)]  # Right, Down, Left, Up, Stay

    def get_next_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.choice(range(5))
        else:
            return np.argmax(self.q_table[state[0], state[1]])

    def take_action(self, action):
        next_position = (self.current_position[0] + self.actions[action][0],
                         self.current_position[1] + self.actions[action][1])
        if (0 <= next_position[0] < self.grid_size[0] and
            0 <= next_position[1] < self.grid_size[1] and
            next_position not in self.obstacles):
            return next_position
        return self.current_position

    def update_q_table(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[next_state[0], next_state[1]])
        td_target = reward + self.gamma * self.q_table[next_state[0], next_state[1], best_next_action]
        td_error = td_target - self.q_table[state[0], state[1], action]
        self.q_table[state[0], state[1], action] += self.alpha * td_error

class MultiAgentEnvironment:
    def __init__(self, grid, agents):
        self.grid = grid
        self.agents = agents
        self.initial_positions = [agent.start for agent in agents]  # Store initial positions

    def reset(self):
        for agent, initial_pos in zip(self.agents, self.initial_positions):
            agent.current_position = initial_pos

    def step(self):
        new_positions = {}
        collisions = set()
        actions_taken = {}

        # First pass: Check next positions for potential collisions
        for agent in self.agents:
            action = agent.get_next_action(agent.current_position)
            next_position = agent.take_action(action)

            if next_position in new_positions:
                collisions.add(next_position)
            else:
                new_positions[next_position] = agent
                actions_taken[agent.agent_id] = next_position

        # Second pass: Move agents, handle collisions
        for agent in self.agents:
            if agent.agent_id in actions_taken:
                next_position = actions_taken[agent.agent_id]

                if next_position in collisions:
                    # Collision detected, stop this agent from moving
                    reward = -25  # Collision penalty
                    next_position = agent.current_position  # Stay in place
                elif next_position == agent.goal:
                    reward = 10  # Goal reward
                else:
                    reward = -1  # Step penalty
                
                agent.update_q_table(agent.current_position, agent.get_next_action(agent.current_position), reward, next_position)
                agent.current_position = next_position
            else:
                # This agent did not get to move (it is colliding)
                agent.update_q_table(agent.current_position, agent.get_next_action(agent.current_position), -5, agent.current_position)  # Staying in place penalty

        return [agent.current_position for agent in self.agents]

# The rest of your BotGrid and other classes can remain the same.




class BotGrid(tk.Tk):
    def __init__(self, grid, agents, env):
        super().__init__()
        self.title("Multi-Agent Q-Learning with A* Path Planning")

        self.grid = grid
        self.agents = agents
        self.env = env

        self.rows = len(grid)
        self.cols = len(grid[0])
        self.cell_size = 100

        self.canvas = tk.Canvas(self, width=self.cols * self.cell_size, height=self.rows * self.cell_size)
        self.canvas.pack()

        self.create_grid()
        self.bot_squares = self.create_bots()
        self.individual_steps = [0] * len(self.agents)
        self.start_time = time.time()
        self.steps = [0 for _ in agents]
        self.steps_array = [0] * len(agents) 
        self.timer_label = tk.Label(self, text="Time: 0s")
        self.timer_label.pack()
        # self.steps_label = tk.Label(self, text=", ".join([f"Bot {i} steps: 0" for i in range(len(agents))]))
        # self.steps_label.pack()

        # self.avg_steps_label = tk.Label(self, text="Average Steps: 0.00")
        # self.avg_steps_label.pack()
        # Reset agents to their initial positions before starting visualization
        self.env.reset()

        self.move_bots()

    def calculate_average_steps(self):
        if len(self.steps) == 0:  # Avoid division by zero
            avg_steps = 0
        else:
            avg_steps = sum(self.steps) / len(self.steps)
            print(f"Average is: {avg_steps}")
        

    def create_grid(self):
        for r in range(self.rows):
            for c in range(self.cols):
                x1, y1 = c * self.cell_size, r * self.cell_size
                x2, y2 = x1 + self.cell_size, y1 + self.cell_size
                color = "white" if self.grid[r][c] == 1 else "black"
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color)

    def create_bots(self):
        bot_squares = []
        colors = ['red', 'blue', 'green', 'yellow', 'purple', 'orange']
        for i, agent in enumerate(self.agents):
            r, c = agent.current_position
            x1, y1 = c * self.cell_size, r * self.cell_size
            x2, y2 = x1 + self.cell_size, y1 + self.cell_size
            color = colors[i % len(colors)]
            bot_square = self.canvas.create_rectangle(x1, y1, x2, y2, fill=color)
            bot_squares.append(bot_square)
        return bot_squares
    def move_bots(self):
        def move_step():
            new_positions = self.env.step()
            
            
            
            # Create a dynamic list to hold individual step counts
            
              # Initialize with zero for each bot

            for i, new_pos in enumerate(new_positions):

                if self.agents[i].current_position == self.agents[i].goal:
                    # print(f"Bot {i} has reached the goal at {self.agents[i].goal}.")
                    if(self.steps[i] == 0):
                        self.steps[i] = self.individual_steps[i] + 1
                # Check if the bot has moved
                if new_pos != self.agents[i].current_position:
                    self.move_bot(i, new_pos)
                    # self.steps[i] += 1  # Increment total steps for a move
                    self.individual_steps[i] += 1  # Update individual steps
                    # print(f"Bot {i} moves to {new_pos}")
                else:  # If bot doesn't move, keep its position the same
                    self.move_bot(i, self.agents[i].current_position)
                    # self.steps[i] += 1  # Increment total steps for staying in place
                    self.individual_steps[i] += 1  # Update individual steps
                    # print(f"Bot {i} waits at {self.agents[i].current_position}")

            elapsed_time = time.time() - self.start_time
            self.timer_label.config(text=f"Time: {int(elapsed_time)}s")
        
            # self.steps_label.config(text=", ".join([f"Bot {i} steps: {self.individual_steps[i]}" for i in range(len(self.agents))]))

            if not all(agent.current_position == agent.goal for agent in self.agents):
                self.after(3000, move_step)
            else:
                self.calculate_average_steps()
                # print(f"Average Steps: {avg_steps:.2f}")
                # self.avg_steps_label.config(text=f"Average Steps: {avg_steps:.2f}")

                # Print the steps for each bot, using the dynamic list
                print(f'Total time taken:  {elapsed_time} seconds')
                for i, steps in enumerate(self.steps):
                    print(f"Bot {i} individual steps this round: {steps}")

        move_step()
    def move_bot(self, bot_index, new_position):
        r, c = new_position
        x1, y1 = c * self.cell_size, r * self.cell_size
        x2, y2 = x1 + self.cell_size, y1 + self.cell_size
        self.canvas.coords(self.bot_squares[bot_index], x1, y1, x2, y2)

def run_hybrid_q_learning_demo():
    # User input for grid size
    rows = int(simpledialog.askstring("Grid Size", "Enter number of rows:"))
    cols = int(simpledialog.askstring("Grid Size", "Enter number of columns:"))

    # Initialize the grid
    grid = [[1 for _ in range(cols)] for _ in range(rows)]

    # Get obstacles from the user
    obstacle_input = simpledialog.askstring("Obstacles Input", "Enter obstacles in the form 'row,col' (e.g., '1,1;2,3'):").strip()
    obstacles = set()
    if obstacle_input:
        for obs in obstacle_input.split(';'):
            r, c = map(int, obs.split(','))
            grid[r][c] = 0
            obstacles.add((r, c))

    # Get bot configurations from the user
    num_bots = int(simpledialog.askstring("Bots Input", "Enter number of bots:"))
    agents = []
    for i in range(num_bots):
        start = tuple(map(int, simpledialog.askstring("Bot Input", f"Enter start position for Bot {i + 1} (row,col):").split(',')))
        goal = tuple(map(int, simpledialog.askstring("Bot Input", f"Enter goal position for Bot {i + 1} (row,col):").split(',')))
        
        # Generate initial path using A*
        initial_path = a_star_search(grid, start, goal)
        
        agent = QLearningAgent(i, (rows, cols), obstacles, start, goal, initial_path)
        agents.append(agent)

    env = MultiAgentEnvironment(grid, agents)

    # Train agents
    print("Training agents...")
    for episode in range(6000):
        env.reset()  # Reset to initial positions at the start of each episode
        while not all(agent.current_position == agent.goal for agent in agents):
            env.step()

    # Reset agents to initial positions for visualization
    env.reset()
    print("Visualizing the grid and paths...")
    app = BotGrid(grid, agents, env)
    app.mainloop()

if __name__ == "__main__":
    run_hybrid_q_learning_demo()

Training agents...
Visualizing the grid and paths...
Average is: 7.833333333333333
Total time taken:  66.229820728302 seconds
Bot 0 individual steps this round: 23
Bot 1 individual steps this round: 2
Bot 2 individual steps this round: 2
Bot 3 individual steps this round: 8
Bot 4 individual steps this round: 9
Bot 5 individual steps this round: 3
